In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import plotly.figure_factory as ff
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [91]:
import numpy as np
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import plotly.figure_factory as ff
import os
import plotly.express as px
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import dash_bootstrap_components as dbc
import pymongo
from bson.json_util import loads, dumps
mongo_username = os.getenv('MONGO_INITDB_ROOT_USERNAME')
mongo_password = os.getenv('MONGO_INITDB_ROOT_PASSWORD')
mongo_init_db = os.getenv('MONGO_INITDB_DATABASE')
import wordcloud
import matplotlib.pyplot as plt

In [2]:
covid_data = pd.read_csv("owid-covid-data.csv")

In [3]:
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [4]:
# Get the list of column names
column_names = covid_data.columns
print(column_names)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [5]:
# 1. Data Summary
print(covid_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355599 entries, 0 to 355598
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    355599 non-null  object 
 1   continent                                   338653 non-null  object 
 2   location                                    355599 non-null  object 
 3   date                                        355599 non-null  object 
 4   total_cases                                 317456 non-null  float64
 5   new_cases                                   345855 non-null  float64
 6   new_cases_smoothed                          344596 non-null  float64
 7   total_deaths                                295737 non-null  float64
 8   new_deaths                                  345907 non-null  float64
 9   new_deaths_smoothed                         344677 non-null  float64
 

In [6]:
##Data Summary
print(covid_data.describe())

        total_cases     new_cases  new_cases_smoothed  total_deaths  \
count  3.174560e+05  3.458550e+05        3.445960e+05  2.957370e+05   
mean   6.796974e+06  9.457200e+03        9.491605e+03  8.666838e+04   
std    4.123893e+07  1.094194e+05        9.374232e+04  4.426444e+05   
min    1.000000e+00  0.000000e+00        0.000000e+00  1.000000e+00   
25%    8.285000e+03  0.000000e+00        2.860000e-01  1.280000e+02   
50%    7.143600e+04  1.000000e+00        2.357100e+01  1.341000e+03   
75%    7.667340e+05  2.510000e+02        4.821430e+02  1.198800e+04   
max    7.718202e+08  8.401960e+06        6.402036e+06  6.978162e+06   

         new_deaths  new_deaths_smoothed  total_cases_per_million  \
count  345907.00000        344677.000000            317456.000000   
mean       84.23524            84.532007            103457.466834   
std       608.98649           554.835927            152731.556394   
min         0.00000             0.000000                 0.000000   
25%         0.0

In [7]:
missing_values_count = covid_data.isnull().sum()
print(missing_values_count)


iso_code                                        0
continent                                   16946
location                                        0
date                                            0
total_cases                                 38143
                                            ...  
population                                      0
excess_mortality_cumulative_absolute       343388
excess_mortality_cumulative                343388
excess_mortality                           343388
excess_mortality_cumulative_per_million    343388
Length: 67, dtype: int64


In [8]:

missing_values_percentage = (covid_data.isnull().sum() / len(covid_data)) * 100
#pd.set_option('display.max_rows', 50)
print(missing_values_percentage)


iso_code                                    0.000000
continent                                   4.765480
location                                    0.000000
date                                        0.000000
total_cases                                10.726408
                                             ...    
population                                  0.000000
excess_mortality_cumulative_absolute       96.566076
excess_mortality_cumulative                96.566076
excess_mortality                           96.566076
excess_mortality_cumulative_per_million    96.566076
Length: 67, dtype: float64


Data cleaning Stage. Consider converting the data from object (String) into
datetime format

In [9]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [10]:
covid_data['date']

0        2020-01-03
1        2020-01-04
2        2020-01-05
3        2020-01-06
4        2020-01-07
            ...    
355594   2023-11-05
355595   2023-11-06
355596   2023-11-07
355597   2023-11-08
355598   2023-11-09
Name: date, Length: 355599, dtype: datetime64[ns]

checking to drop duplicates in the dataset

In [11]:
covid_data = covid_data.drop_duplicates()

In [12]:
##Removing missing values above 50 percent from the dataset

#missing_values_percentage = covid_data.isnull().mean() * 100

# Identify columns where the percentage of missing values is greater than 50%
columns_to_drop = missing_values_percentage[missing_values_percentage > 50].index

# Drop these columns from the dataset
covid_data_cleaned = covid_data.drop(columns=columns_to_drop)

In [13]:
# cols_to_fill_zero = ['total_cases', 'new_cases', 'total_deaths', 'new_deaths']
# covid_data_cleaned[cols_to_fill_zero] = covid_data_cleaned[cols_to_fill_zero].fillna(0)
covid_data_cleaned

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,population
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,1.337,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,41128772.0
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,1.337,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,41128772.0
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,1.337,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,41128772.0
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,1.337,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,41128772.0
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,1.337,1803.987,597.029,9.59,NaN,NaN,0.5,64.83,0.511,41128772.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355594,ZWE,Africa,Zimbabwe,2023-11-05,265848.0,0.0,0.0,5723.0,0.0,0.0,...,1.882,1899.775,307.846,1.82,1.6,30.7,1.7,61.49,0.571,16320539.0
355595,ZWE,Africa,Zimbabwe,2023-11-06,265848.0,0.0,0.0,5723.0,0.0,0.0,...,1.882,1899.775,307.846,1.82,1.6,30.7,1.7,61.49,0.571,16320539.0
355596,ZWE,Africa,Zimbabwe,2023-11-07,265848.0,0.0,0.0,5723.0,0.0,0.0,...,1.882,1899.775,307.846,1.82,1.6,30.7,1.7,61.49,0.571,16320539.0
355597,ZWE,Africa,Zimbabwe,2023-11-08,265848.0,0.0,0.0,5723.0,0.0,0.0,...,1.882,1899.775,307.846,1.82,1.6,30.7,1.7,61.49,0.571,16320539.0


In [14]:
# pd.set_option('display.max_rows', 50)
# covid_data_cleaned[cols_to_fill_zero].head(30)

In [15]:
##Checking for anomalies. Thus ensuring there are no negative values in columns 
##where negatives wont make sense by clipping them to zero

cols_to_check = ['total_cases', 'new_cases', 'total_deaths', 'new_deaths']
covid_data_cleaned[cols_to_check] = covid_data_cleaned[cols_to_check].clip(lower=0)


In [16]:
###Checking the columns for the cleaned dataset

cleaned_column_names = covid_data_cleaned.columns
print(cleaned_column_names)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate',
       'new_vaccinations_smoothed', 'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed',
       'new_people_vaccinated_smoothed_per_hundred', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population'],
      dtype='object')


In [17]:
#Exporting the cleaned dataset to a new CSV file
cleaned_file_path = 'cleaned_covid_data.csv' 
covid_data_cleaned.to_csv(cleaned_file_path, index=False)


In [21]:
# # Time series plot for total cases
# plt.figure(figsize=(8, 5))
# sns.lineplot(data=covid_data_cleaned, x='date', y='total_cases')
# plt.title('Total COVID-19 Cases Over Time')
# plt.show()

In [19]:
# # COVID-19 cases by continent
# plt.figure(figsize=(10, 6))
# sns.barplot(x='continent', y='total_cases', data=covid_data_cleaned)
# plt.title('Total COVID-19 Cases by Continent')
# plt.show()

In [ ]:
# # Histogram for GDP per capita
# plt.figure(figsize=(10, 6))
# sns.histplot(covid_data_cleaned['gdp_per_capita'], kde=True)
# plt.title('Distribution of GDP per Capita')
# plt.show()

In [55]:
# Creating the country_profile DataFrame
country_profile_columns = [
    'iso_code', 'continent', 'location', 'population', 'gdp_per_capita', 
    'aged_65_older', 'aged_70_older', 'cardiovasc_death_rate', 
    'diabetes_prevalence', 'female_smokers', 'male_smokers', 
    'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 
    'population_density', 'median_age'
]

# Dropping duplicates since we only need unique values for each country
country_profile_df = covid_data_cleaned[country_profile_columns].drop_duplicates()


In [56]:
# Creating the covid_daily_stats DataFrame
covid_daily_stats_columns = [
    'iso_code', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 
    'total_deaths', 'new_deaths', 'new_deaths_smoothed', 
    'total_cases_per_million', 'new_cases_per_million', 
    'new_cases_smoothed_per_million', 'total_deaths_per_million', 
    'new_deaths_per_million', 'new_deaths_smoothed_per_million', 
    'reproduction_rate'
]

covid_daily_stats_df = covid_data_cleaned[covid_daily_stats_columns].drop_duplicates()


In [57]:
# Creating the covid_vaccination_stats DataFrame
covid_vaccination_stats_columns = [
    'iso_code', 'date', 'new_vaccinations_smoothed', 
    'new_vaccinations_smoothed_per_million', 'new_people_vaccinated_smoothed', 
    'new_people_vaccinated_smoothed_per_hundred'
]

covid_vaccination_stats_df = covid_data_cleaned[covid_vaccination_stats_columns].drop_duplicates()


In [58]:
# Creating the government_response DataFrame
government_response_columns = ['iso_code', 'date', 'stringency_index']

government_response_df = covid_data_cleaned[government_response_columns].drop_duplicates()


In [41]:
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}/{db}".format (
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    db = 'covid'
))

In [42]:
dbserver = psycopg2.connect(
     user = 'postgres',
     password = POSTGRES_PASSWORD,
     host = 'postgres',
     #host = 'host.docker.internal',
     port= '5432'   
)
dbserver.autocommit = True

In [43]:
cursor = dbserver.cursor()

In [62]:
# try:
#     cursor.execute('CREATE DATABASE covid')
# except:
#     cursor.execute('DROP DATABASE covid')
#     cursor.execute('CREATE DATABASE covid')

In [49]:
cursor.execute("SELECT datname FROM pg_database")


In [50]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

In [53]:
# Execute a query to list all databases
cursor.execute("SELECT datname FROM pg_database")

    # Fetch and print the results
databases = cursor.fetchall()
for db in databases:
    print(db[0])

postgres
template1
template0
congress
cardib
nba
covid


In [68]:
country_profile_df.to_sql('country_profile', engine, chunksize =1000, if_exists='replace', index=False)

255

In [69]:
covid_daily_stats_df.to_sql('covid_daily_stats_df', engine,  chunksize =1000, if_exists='replace', index=False)

355599

In [70]:
covid_vaccination_stats_df.to_sql('covid_vaccination_stats_df', engine,  chunksize =1000, if_exists='replace', index=False)


355599

In [71]:

government_response_df.to_sql('government_response_df', engine,  chunksize =1000, if_exists='replace', index=False)


355599

In [77]:
myquery = '''

SELECT *
FROM country_profile

'''
pd.read_sql_query(myquery, con= engine)


,iso_code,continent,location,population,gdp_per_capita,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,population_density,median_age
0,AFG,Asia,Afghanistan,4.112877e+07,1803.987,2.581,1.337,597.029,9.59,NaN,NaN,0.500,64.83,0.511,54.422,18.6
1,OWID_AFR,None,Africa,1.426737e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2.842318e+06,11803.431,13.188,8.643,304.195,10.08,7.100,51.200,2.890,78.57,0.795,104.871,38.0
3,DZA,Africa,Algeria,4.490323e+07,13913.839,6.211,3.857,278.364,6.73,0.700,30.400,1.900,76.88,0.748,17.348,29.1
4,ASM,Oceania,American Samoa,4.429500e+04,NaN,NaN,NaN,283.750,NaN,NaN,NaN,NaN,73.74,NaN,278.205,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,ESH,Africa,Western Sahara,5.760050e+05,NaN,NaN,1.380,NaN,NaN,NaN,NaN,NaN,70.26,NaN,NaN,28.4
251,OWID_WRL,None,World,7.975105e+09,15469.207,8.696,5.355,233.070,8.51,6.434,34.635,2.705,72.58,0.737,58.045,30.9
252,YEM,Asia,Yemen,3.369661e+07,1479.147,2.922,1.583,495.003,5.35,7.600,29.200,0.700,66.12,0.470,53.508,20.3
253,ZMB,Africa,Zambia,2.001767e+07,3689.251,2.480,1.542,234.499,3.94,3.100,24.700,2.000,63.89,0.584,22.995,17.7


In [79]:
myquery = '''

SELECT *
FROM government_response_df

'''
pd.read_sql_query(myquery, con= engine)


,iso_code,date,stringency_index
0,AFG,2020-01-03,0.0
1,AFG,2020-01-04,0.0
2,AFG,2020-01-05,0.0
3,AFG,2020-01-06,0.0
4,AFG,2020-01-07,0.0
...,...,...,...
355594,ZWE,2023-11-05,NaN
355595,ZWE,2023-11-06,NaN
355596,ZWE,2023-11-07,NaN
355597,ZWE,2023-11-08,NaN


In [83]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [100]:
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # App layout
# app.layout = html.Div([
#     html.H1("Country Profile Dashboard"),
#     dcc.Graph(id='graph'),
#     html.Button('Update Data', id='btn-update', n_clicks=0)
# ])

# # Callback to update graph
# @app.callback(
#     Output('graph', 'figure'),
#     [Input('btn-update', 'n_clicks')]
# )
# def update_graph(n_clicks):
#     # SQL query to fetch data
#     sql_query = """
#     SELECT * FROM country_profile
#     """
#     df = pd.read_sql_query(sql_query, engine)

#     # Creating a visualization (example: scatter plot of GDP per capita vs life expectancy)
#     fig = px.scatter(df, x='gdp_per_capita', y='life_expectancy', 
#                      hover_data=['location'], color='continent')
    
#     return fig





# if __name__== "__main__":
#     app.run(mode= 'external', host = "0.0.0.0", port = 8050, debug=True)


In [99]:
# Fetching unique country names for dropdown
country_query = "SELECT DISTINCT continent FROM country_profile"
country_df = pd.read_sql_query(country_query, engine)
country_options = [{'label': country, 'value': country} for country in country_df['continent'] if country != None ]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = html.Div([
    html.H1("Country Profile Dashboard"),
    dcc.Dropdown(country_options, id='country-dropdown'),
    dcc.Graph(id='graph'),
    
    
    html.Div(id='dd-output-container')
])

# Callback for updating the graph based on country selection
@app.callback(
    Output('graph', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):
    if selected_country:
        sql_query = f"""
        SELECT * FROM country_profile WHERE continent = '{selected_country}'
        """
    else:
        sql_query = "SELECT * FROM country_profile"
   
    df = pd.read_sql_query(sql_query, engine)

    # Creating a visualization
    fig = px.scatter(df, x='gdp_per_capita', y='life_expectancy',
                     hover_data=['location'], color='continent')
   
    return fig

# Run the app
if __name__== "__main__":
    app.run(mode= 'external', host = "0.0.0.0", port = 8050, debug=True)
